### Docs
* https://developers.google.com/youtube/v3/docs

In [ ]:
# -*- coding: utf-8 -*-
import os, json, flask, requests, pandas as pd

import google.oauth2.credentials
import google_auth_oauthlib.flow
import googleapiclient.discovery
from googleapiclient.http import MediaFileUpload 

In [ ]:
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'
with open('oauth2_user_credentials.json') as fr:
    credentials = google.oauth2.credentials.Credentials(**json.load(fp=fr))
youtube = googleapiclient.discovery.build(API_SERVICE_NAME, 
                                          API_VERSION, 
                                          credentials=credentials)

In [ ]:
df = pd.read_csv('audio/list.csv')
df = df.set_index('vid') 
df = df[df['t_end'].notna()]
print(df.index)
display(df)

In [ ]:
channel_list = youtube.channels().list(
    mine=True,
    part='contentDetails'
  ).execute()

relatedPlaylists = channel_list['items'][0]['contentDetails']['relatedPlaylists']['uploads']

playlist = youtube.playlistItems().list(
    playlistId=relatedPlaylists,
    part='snippet',
    maxResults=5
)

myUploadedVideo = {}
while playlist:
    response = playlist.execute()
    
    # Print information about each video.
    for playlist_item in response['items']:
        title = playlist_item['snippet']['title']
        video_id = playlist_item['snippet']['resourceId']['videoId']
        myUploadedVideo[title]=video_id 

    playlist = youtube.playlistItems().list_next(playlist, response)

In [ ]:
playlistId = open('playerlist_id.txt').read() # my is "PLUenxxxx_xxxxfrNm92hxxxxQdvukmViZ"
for vid, row in df.iterrows(): 
    audio_file=f'video/{vid}.mp4'
    title   = '梁文道 八分 ' + row['title']
    if title in myUploadedVideo: 
        print('[Video Is Already in My Channel]', title)
        continue
    if not os.path.exists(audio_file):
        print('[Video File Does not Exist]', audio_file)
        continue
    print(f'Uploading {title} ... ...')
    continue
    body={
        'snippet':{
            'title':title,
            #'description':'More words for this video',
            'tags':['梁文道','八分'],
            'categoryId':'22' # https://developers.google.com/youtube/v3/docs/videoCategories/list
        },
        'status':{
            'privacyStatus':'public', # ['private','public','unlisted']
            'selfDeclaredMadeForKids':False
        }
    }
    
    uploadedVideo = youtube.videos().insert(
        part=','.join(body.keys()),
        body=body,
        media_body=MediaFileUpload(audio_file, chunksize=-1, resumable=False)
    ).execute()
    
    videoId   = uploadedVideo['id'] 
    
    body = {'snippet':{
        "playlistId": playlistId, # '梁文道－八分'
        #"position": 0,
        "resourceId": {
          "kind": "youtube#video",
          "videoId": videoId
        }
    }}
    
    resource = youtube.playlistItems().insert(part='snippet', body=body).execute()
    print(f'{title} Uploaded!')